In [ ]:
import joblib          
import pandas as pd        
import numpy as np         
from sklearn.metrics import mean_squared_error, mean_absolute_error
from typing import Dict 
import matplotlib.pyplot as plt
from scripts.evaluation import *
import os
import xarray as xr
# import xgboost as xgb      

In [ ]:
LC_MAPPING: Dict[int, str] = {
    10: 'Tree cover',             # Dunkelgrün
    20: 'Shrubland',              # Helleres Grün
    30: 'Grassland',              # Helles leuchtendes Grün
    40: 'Cropland',               # Bräunlich/Beige
    50: 'Built-up',               # Dunkelgrau
    60: 'Bare/Sparse vegetation', # Mittelgrau
    70: 'Snow and ice',           # Sehr helles Weißblau
    80: 'Permanent water bodies', # Kräftiges Blau
    90: 'Herbaceous wetland',     # Mittelgrün
    100: 'Moss and lichen',       # Silber/Grau
    1: 'Unknown/No Data',          # Weiß
}

## Define test data and target var

In [ ]:
test_keys = ["ds_5", "ds_10"]
target_var = "target_basic"
ndvi_input = "NDVI_baisc_t"

path = "../processed_data_final/"
cubes = {}

for key in test_keys:
    load_path = os.path.join(path,key)
    cubes[key] = xr.open_zarr(load_path)

## Get test data

In [ ]:
df_test_master = pd.read_parquet(f"new_data/master_test_combined.parquet").dropna(subset=[target_var]) 

In [ ]:
# 3. Modelle evaluieren
model_dir = "sar_experiments/"
models = [m for m in os.listdir(model_dir) if m.endswith(".pkl")]
all_model_results = []
ts_metrics_storage = {}

# models = ["Model_Basic_Baseline SAR.pkl", "Model_Basic_Baseline Spectral.pkl", "Model_Basic_Temp_Only_with_Lags.pkl","Model_Basic_Temp_Only.pkl" ,"Model_Basic_Water_Only.pkl", "Model_Basic_Water_Only_with_Lags.pkl", "Model_Basic_Full_Climate.pkl"]

for model_file in models:
    pkg = joblib.load(os.path.join(model_dir, model_file))
    model, feats, m_name, target_v = pkg["model"], pkg["features"], pkg["model_name"] , pkg["target_variable"]
    ndvi_in = "NDVI_basic_t" if "basic" in target_v else "NDVI_strict_t"

    print("##"*12)
    print(f"Modell '{m_name}' erfolgreich geladen.")
    print(f"Features im Modell: {feats}")

    # Prediction
    X_eval = df_test_master[feats].copy()
    if "lc_class" in X_eval.columns: X_eval["lc_class"] = X_eval["lc_class"].astype("category")
    df_test_master["preds"] = model.predict(X_eval)
    valid_res = df_test_master.dropna(subset=[target_v]).copy()

    # Metriken berechnen
    global_m = get_metrics(valid_res, target_v)
    m_clear, m_cloudy = evaluate_with_gap_analysis(valid_res, target_v, ndvi_in)
    
    # Recovery Analyse (Aggregiert über Test Cubes)
    recovery_list = []
    for k in test_keys:
        precip_dt = pd.to_datetime(cubes[k].precip_end_date)
        t_times = pd.to_datetime(cubes[k].time_sentinel_2_l2a.values)
        in_idx = np.where(t_times <= precip_dt)[0][-1]
        recovery_list.append(valid_res[(valid_res['cube_origin'] == k) & (valid_res['timestep'] == in_idx)])
    
    m_recov = get_metrics(pd.concat(recovery_list), target_v)
    
    # Zeitverlauf speichern
    if m_name not in ts_metrics_storage:
        ts_metrics_storage[m_name] = {}

    for k in test_keys:
        # Berechne Metriken nur für diesen spezifischen Cube
        cube_res = valid_res[valid_res['cube_origin'] == k]
        if len(cube_res) > 0:
            ts_metrics_storage[m_name][k] = cube_res.groupby("timestep").apply(lambda x: get_metrics(x, target_v))

    # Print-Ausgabe
    print(f"\n▶ {m_name.upper()} | Target: {target_v}")

    print("\n" + "="*40)
    print(f" GLOBALE ERGEBNISSE: {m_name}")
    print("="*40)
    print(f"Anzahl Pixel: {global_m['Count']}")
    print(f"RMSE:         {global_m['RMSE']:.4f}")
    print(f"MAE:          {global_m['MAE']:.4f}")
    print(f"Bias:         {global_m['Bias']:.4f}")


    print("\n" + "="*40)
    print(f"\n--- GAP ANALYSIS: {m_name} ---")
    print("\n" + "="*40)
    print(f"CLEAR SKY (NDVI_t vorhanden):  RMSE: {m_clear['RMSE']:.4f} MAE: {m_clear['MAE']:.4f} (n={int(m_clear['Count'])})")
    print(f"CLOUDY    (Radar-Only):        RMSE: {m_cloudy['RMSE']:.4f} MAE: {m_cloudy['MAE']:.4f} (n={int(m_cloudy['Count'])})")

    print("\n" + "="*40)
    print(f" RECOVERY: {m_name}")
    print("\n" + "="*40)
    print(f"Anzahl Pixel: {m_recov['Count']}")
    print(f"RMSE:         {m_recov['RMSE']:.4f}")
    print(f"MAE:          {m_recov['MAE']:.4f}")
    print(f"Bias:         {m_recov['Bias']:.4f}")
    print("="*50)
    # print(f"  GLOBAL:    RMSE: {global_m['RMSE']:.4f} | Bias: {global_m['Bias']:.4f} \n")
    # print(f"  RECOVERY:  RMSE: {m_recov['RMSE']:.4f} (Moment nach Regen) \n")
    # print(f"  GAP:       Clear: {m_clear['RMSE']:.4f} | Cloudy: {m_cloudy['RMSE']:.4f} \n")


    fi_results = plot_feature_importance(pkg)


    all_model_results.append({
        "Model": m_name, "Target": target_v, "RMSE_Global": global_m["RMSE"],
        "RMSE_Recovery": m_recov["RMSE"], "RMSE_Cloudy": m_cloudy["RMSE"]
    })

df_comparison = pd.DataFrame(all_model_results).sort_values("RMSE_Global")
display(df_comparison)

In [ ]:
df_comparison.to_csv("sar_experiments/model_comparison_df.csv", index=False)

In [ ]:
df_comparison.sort_values("RMSE_Global")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_sar_correlation(df):
    # 1. Auswahl der relevanten SAR-Spalten
    sar_cols = ['vv', 'vh', 'VHVVR', 'VVVHS', 'DpRVIVV', "target_basic", "target_strict"]
    
    # Sicherstellen, dass nur vorhandene Spalten genutzt werden
    existing_cols = [c for c in sar_cols if c in df.columns]
    
    # 2. Korrelation berechnen (Pearson)
    corr_matrix = df[existing_cols].corr()

    # 3. Plotten
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, vmin=-1, vmax=1, fmt='.2f')
    plt.title("Korrelation zwischen SAR-Bändern und Indizes", fontsize=14)
    plt.show()

# Anwendung auf dein Test-Set oder Training-Set
plot_sar_correlation(df_test_master)

In [ ]:
def plot_temporal_performance_per_cube(cubes, ts_metrics_storage, metric="RMSE"):
    test_keys = list(cubes.keys())
    
    for k in test_keys:
        ds = cubes[k]
        fig, ax = plt.subplots(figsize=(14, 6))
        
        # Zeitachse vorbereiten (Mapping von ID zu Datum)
        t_times = pd.to_datetime(ds.time_sentinel_2_l2a.values)
        
        # Für jedes Modell die Linie zeichnen
        for m_name in ts_metrics_storage.keys():
            if k in ts_metrics_storage[m_name]:
                stats = ts_metrics_storage[m_name][k]
                # Wir mappen die Timestep-Indizes auf die echten Daten
                plot_dates = t_times[stats.index.values]
                ax.plot(plot_dates, stats[metric], label=m_name, marker='o', markersize=4, alpha=0.8)

        # Perioden aus dem Cube holen
        start_d = pd.to_datetime(ds.attrs["drought_start_date"])
        end_d = pd.to_datetime(ds.attrs["drought_end_date"])
        start_p = pd.to_datetime(ds.attrs["precip_start_date"])
        end_p = pd.to_datetime(ds.attrs["precip_end_date"])

        # Perioden farblich markieren
        ax.axvspan(start_d, end_d, color="red", alpha=0.1, label="Dürre-Periode")
        ax.axvspan(start_p, end_p, color="blue", alpha=0.1, label="Regen-Periode")

        # Styling
        ax.set_title(f"Performance Verlauf für Cube: {k}", fontsize=15, fontweight='bold')
        ax.set_ylabel(f"{metric} (niedriger ist besser)")
        ax.set_xlabel("Datum")
        ax.grid(True, alpha=0.2)
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
        
        plt.tight_layout()
        plt.show()

# Aufruf
plot_temporal_performance_per_cube(cubes, ts_metrics_storage)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_detailed_model_performance(cubes, ts_metrics_storage):
    """
    Erstellt pro Modell eine Abbildung mit Subplots für jeden Cube.
    Zeigt RMSE und Bias im Zeitverlauf mit Durchschnittslinien.
    """
    metrics = ["RMSE", "Bias"]
    
    for m_name in ts_metrics_storage.keys():
        current_model_data = ts_metrics_storage[m_name]
        n_cubes = len(current_model_data)
        
        # Erstelle Grid: Zeilen = Metriken, Spalten = Cubes
        fig, axes = plt.subplots(len(metrics), n_cubes, figsize=(7 * n_cubes, 10), sharey='row')
        fig.suptitle(f"DETAILLIERTE ANALYSE: {m_name.upper()}", fontsize=18, fontweight='bold', y=1.02)

        for col_idx, (cube_key, stats) in enumerate(current_model_data.items()):
            ds = cubes[cube_key]
            t_times = pd.to_datetime(ds.time_sentinel_2_l2a.values)
            plot_dates = t_times[stats.index.values]
            
            # Event-Daten
            events = {
                "Dürre": (pd.to_datetime(ds.attrs["drought_start_date"]), 
                          pd.to_datetime(ds.attrs["drought_end_date"]), "red"),
                "Regen": (pd.to_datetime(ds.attrs["precip_start_date"]), 
                          pd.to_datetime(ds.attrs["precip_end_date"]), "blue")
            }

            for row_idx, metric in enumerate(metrics):
                # Handle Fall für nur einen Cube (axes ist dann 1D)
                ax = axes[row_idx, col_idx] if n_cubes > 1 else axes[row_idx]
                
                # 1. Zeitverlauf plotten
                ax.plot(plot_dates, stats[metric], marker='o', markersize=3, label=f"{metric} Verlauf")
                
                # 2. Durchschnittslinie hinzufügen
                mean_val = stats[metric].mean()
                ax.axhline(mean_val, color='darkorange', linestyle='--', linewidth=2, 
                           label=f"Ø-{metric}: {mean_val:.4f}")
                
                # 3. Referenzlinie für Bias bei 0
                if metric == "Bias":
                    ax.axhline(0, color='black', linewidth=1, alpha=0.5)

                # 4. Phasen markieren
                for label, (start, end, color) in events.items():
                    ax.axvspan(start, end, color=color, alpha=0.1)

                # Styling
                if row_idx == 0:
                    ax.set_title(f"Cube: {cube_key}", fontsize=14, fontweight='bold')
                if col_idx == 0:
                    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
                
                ax.grid(True, alpha=0.2)
                ax.legend(loc='best', fontsize='small')

        plt.tight_layout()
        plt.show()

# Anwendung
plot_detailed_model_performance(cubes, ts_metrics_storage)

In [ ]:
model_dir = "models_new/"
models = os.listdir(model_dir)
for model in models:

    model_name  = model.split(".")[0]

    # Get model and info
    model_path = os.path.join(model_dir, model)
    model_package = joblib.load(model_path)
    model = model_package["model"]
    features_from_model = model_package["features"]
    model_name = model_package["model_name"]

    print("##"*12)
    print(f"Modell '{model_name}' erfolgreich geladen.")
    print(f"Features im Modell: {features_from_model}")

    # Restrict to relevant features
    X_eval = df_eval[features_from_model].copy()

    # Explicitly convert lc_class to category
    if "lc_class" in X_eval.columns:
        X_eval["lc_class"] = X_eval["lc_class"].astype("category")

    # Prediction
    df_eval["preds"] = model.predict(X_eval)

    # Drop all lines where target_var is NAN
    valid_results = df_eval.dropna(subset=[target_var]).copy()

    # Map lc_class to LC name
    valid_results['LC_Name'] = valid_results['lc_class'].map(LC_MAPPING)

    print(f"Prediction abgeschlossen. Evaluierung auf {len(valid_results):,} validen Zielwerten.")


    # Globale Metriken
    global_stats = get_metrics(valid_results)
    print("\n" + "="*40)
    print(f" ERGEBNISSE: {model_name}")
    print("="*40)
    print(global_stats.to_string())
    
    # Nach Landcover
    lc_stats = valid_results.groupby("LC_Name", observed=True).apply(get_metrics)
    print("\n" + "="*40)
    print(" NACH LANDCOVER-KLASSE")
    print("="*40)
    print(lc_stats.to_string())


    # Get metrics for cloudy vs clear input
    stats_clear, stats_cloudy = evaluate_with_gap_analysis(valid_results, model_name)

    fi_results = plot_feature_importance(model_package)

    


In [ ]:
def get_metrics(df):
    y_true = df[target_var]
    y_pred = df["preds"]
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    bias = np.mean(y_pred - y_true)
    return pd.Series({"RMSE": rmse, "MAE": mae, "Bias": bias, "Count": len(df)})

# Globale Metriken
global_stats = get_metrics(valid_results)
print("\n" + "="*40)
print(f" ERGEBNISSE: {model_name}")
print("="*40)
print(global_stats.to_string())

# Nach Landcover
lc_stats = valid_results.groupby("LC_Name", observed=True).apply(get_metrics)
print("\n" + "="*40)
print(" NACH LANDCOVER-KLASSE")
print("="*40)
print(lc_stats.to_string())

In [ ]:
def evaluate_with_gap_analysis(df_results, model_label="Model"):
    # Hilfsfunktion für Metriken (falls noch nicht global definiert)
    def calculate_stats(df):
        if len(df) == 0: return pd.Series({"RMSE": np.nan, "MAE": np.nan, "Bias": np.nan, "Count": 0})
        y_true, y_pred = df[target_var], df["preds"]
        return pd.Series({
            "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
            "MAE": mean_absolute_error(y_true, y_pred),
            "Bias": np.mean(y_pred - y_true),
            "Count": int(len(df))
        })

    # Filter auf Zeilen mit Target (sonst kein Fehler berechenbar)
    df_clean = df_results.dropna(subset=[target_var]).copy()

    # Split
    m_clear = calculate_stats(df_clean[df_clean[ndvi_input].notna()])
    m_cloudy = calculate_stats(df_clean[df_clean[ndvi_input].isna()])

    # Schöner Output
    print(f"\n--- GAP ANALYSIS: {model_label} ---")
    print(f"CLEAR SKY (NDVI_t vorhanden):  RMSE: {m_clear['RMSE']:.4f} (n={int(m_clear['Count'])})")
    print(f"CLOUDY    (Radar-Only):        RMSE: {m_cloudy['RMSE']:.4f} (n={int(m_cloudy['Count'])})")
    
    return m_clear, m_cloudy

# Aufruf
stats_clear, stats_cloudy = evaluate_with_gap_analysis(valid_results, model_name)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_feature_importance(model_package, top_n=20):
    """
    Plottet die Feature Importance in absteigender Reihenfolge und 
    gibt die exakten Werte als Tabelle aus.
    """
    model = model_package["model"]
    features = model_package["features"]
    model_name = model_package.get("model_name", "Model")

    # Importance aus dem XGBRegressor extrahieren
    importances = model.feature_importances_
    
    # DataFrame erstellen und sortieren
    fi_df = pd.DataFrame({
        'Feature': features,
        'Importance_Gain': importances
    }).sort_values(by='Importance_Gain', ascending=False).reset_index(drop=True)

    # 1. Plotten
    plt.figure(figsize=(10, 0.4 * min(len(fi_df), top_n) + 2))
    ax = sns.barplot(
        x='Importance_Gain', 
        y='Feature', 
        data=fi_df.head(top_n), 
        palette='magma',
        hue='Feature',
        legend=False
    )
    
    # Werte direkt an die Balken schreiben
    for p in ax.patches:
        width = p.get_width()
        ax.text(width + 0.002, p.get_y() + p.get_height()/2, 
                f'{width:.4f}', va='center', fontsize=10)

    plt.title(f'Feature Importance (Sorted by Gain) - {model_name}', fontsize=14)
    plt.xlabel('Normalized Importance Score')
    plt.ylabel('Features')
    plt.xlim(0, fi_df['Importance_Gain'].max() * 1.15) # Platz für Text rechts
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # 2. Tabellarische Ausgabe
    print(f"\n--- Feature Importance Liste: {model_name} ---")
    print(fi_df.head(top_n).to_string(index=False, formatters={'Importance_Gain': '{:,.4f}'.format}))
    
    return fi_df

# Anwendung:
fi_results = plot_feature_importance(model_package)

In [ ]:
path_dir = "../processed_data/"
data_dir = os.listdir(path_dir)
print(data_dir)

cubes = {}

for path in data_dir:
    key = path[5:9]
    load_path = os.path.join(path_dir, path)
    cubes[key] = xr.open_zarr(load_path)

In [ ]:
import pandas as pd
import numpy as np

def evaluate_post_precip_recovery(df_results, ds_test):
    # 1. Das precip_end_date aus dem Cube holen und konvertieren
    # Wir nehmen an, es ist ein einzelner Wert oder wir nehmen den ersten
    precip_end_dt = pd.to_datetime(ds_test.precip_end_date)
    if isinstance(precip_end_dt, pd.DatetimeIndex):
        precip_end_dt = precip_end_dt[0]
        
    print(f"Referenzdatum (Regenende): {precip_end_dt.date()}")

    # 2. Die Zeitstempel-Liste aus dem Cube holen
    # Damit können wir die 'timestep' ID (0, 1, 2...) wieder in echte Daten umwandeln
    test_times = pd.to_datetime(ds_test.time_sentinel_2_l2a.values)
    
    # 3. Den Index finden, der am nächsten am precip_end_dt liegt (aber davor oder am selben Tag)
    # Das ist der Tag, an dem wir die Features messen (Input t)
    input_idx = np.where(test_times <= precip_end_dt)[0][-1]
    
    # Das Ziel (Target) ist der nächste verfügbare Zeitschritt (t+1)
    target_idx = input_idx + 1
    
    print(f"Input Timestep ID: {input_idx} (Datum: {test_times[input_idx].date()})")
    print(f"Target Timestep ID: {target_idx} (Datum: {test_times[target_idx].date()})")

    # 4. In valid_results genau diese Zeitschritte filtern
    # Wir suchen Zeilen, wo 'timestep' dem input_idx entspricht
    # (Da unser df so gebaut ist, dass bei timestep i der Input i und das Target i+1 ist)
    recovery_df = df_results[df_results['timestep'] == input_idx].copy()

    if recovery_df.empty:
        print("Warnung: Keine Daten für diesen spezifischen Zeitschritt gefunden!")
        return None

    # 5. Metriken für diesen speziellen Moment berechnen
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    
    y_true = recovery_df["target_basic"]
    y_pred = recovery_df["preds"]
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    bias = np.mean(y_pred - y_true)
    
    print("\n" + "="*50)
    print(f"PERFORMANCE DIREKT NACH REGENENDE")
    print("="*50)
    print(f"Anzahl Pixel: {len(recovery_df)}")
    print(f"RMSE:         {rmse:.4f}")
    print(f"MAE:          {mae:.4f}")
    print(f"Bias:         {bias:.4f}")
    print("="*50)
    
    return recovery_df, input_idx

# Anwendung
# valid_results muss aus deiner vorherigen Evaluation stammen
df_recovery, input_idx = evaluate_post_precip_recovery(valid_results, cubes[test_key])

print("I checked: The day before precip_end_date is completely clouded")
import matplotlib.pyplot as plt

# Erstelle eine Figur mit zwei Unterplots (1 Zeile, 2 Spalten)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Linker Plot: Letzter Zeitstempel vor/am Regenende (Input)
input_date = pd.to_datetime(cubes[test_key].time_sentinel_2_l2a.values[input_idx]).date()
cubes[test_key].NDVI_basic.isel(time_sentinel_2_l2a=input_idx).plot(ax=ax1, cmap='RdYlGn', vmin=0, vmax=1)
ax1.set_title(f"Input: Vor/am Regenende\n({input_date})")

# 2. Rechter Plot: Erster Zeitstempel nach Regenende (Target)
target_date = pd.to_datetime(cubes[test_key].time_sentinel_2_l2a.values[input_idx + 1]).date()
cubes[test_key].NDVI_basic.isel(time_sentinel_2_l2a=input_idx + 1).plot(ax=ax2, cmap='RdYlGn', vmin=0, vmax=1)
ax2.set_title(f"Target: Nach Regenende\n({target_date})")

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_error_heatmap(df_eval, ds_test):
    # 1. Timesteps den echten Monaten zuordnen
    # Wir holen die Zeitstempel aus dem ursprünglichen Test-Cube
    time_coords = ds_test.time_sentinel_2_l2a.values
    
    # Mapping-Dictionary: Timestep-Index -> Monat (Name oder Zahl)
    # Da wir t+1 vorhersagen, ist der Monat von 'target' der relevante
    month_map = {i: pd.to_datetime(time_coords[i+1]).month_name() for i in range(len(time_coords)-1)}
    
    df_plot = df_eval.copy()
    df_plot['Month'] = df_plot['timestep'].map(month_map)
    
    # 2. RMSE pro Monat und LC-Klasse berechnen
    def calc_rmse(group):
        valid = group.dropna(subset=['target', 'preds'])
        if len(valid) < 10: return np.nan # Ignoriere zu kleine Stichproben
        return np.sqrt(mean_squared_error(valid['target'], valid['preds']))

    heatmap_data = df_plot.groupby(['lc_class', 'Month']).apply(calc_rmse).unstack()
    
    # Monate sortieren (damit sie nicht alphabetisch Jan, Aug, Dec... sind)
    month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
                   'July', 'August', 'September', 'October', 'November', 'December']
    existing_months = [m for m in month_order if m in heatmap_data.columns]
    heatmap_data = heatmap_data[existing_months]

    # 3. Plotten
    plt.figure(figsize=(14, 8))
    sns.heatmap(heatmap_data, annot=True, cmap='YlOrRd', fmt='.3f', cbar_kws={'label': 'RMSE'})
    
    plt.title('Modell-Fehler (RMSE) nach Vegetationstyp und Monat', fontsize=15)
    plt.xlabel('Monat der Vorhersage', fontsize=12)
    plt.ylabel('Landcover Klasse (ESA LC)', fontsize=12)
    plt.show()

# Heatmap generieren
plot_error_heatmap(df_eval, cubes[test_key])